In [24]:
import pandas as pd
import nltk
import csv

df = pd.read_csv('review_text.csv',encoding='utf8')
docs = df['reviews'].values
data = []
for doc in docs:
    tokens = nltk.word_tokenize(doc)
    pos = nltk.pos_tag(tokens)
    sentence = []
    for item in pos:
        words = {}
        words['token'] = item[0]
        words['pos'] = item[1]
        words['label'] = 'O' #Default BIO tag
        sentence.append(words)
    data.append(sentence)

In [28]:
print(len(data))
a = [2,3,4,5]
a[1:]

898


[3, 4, 5]

In [30]:
import json
with open('reviews-aspect-0-299.json', 'w') as outfile:
    json.dump(data[:300], outfile)
with open('reviews-aspect-300-599.json', 'w') as outfile:
    json.dump(data[300:600], outfile)
with open('reviews-aspect-600.json', 'w') as outfile:
    json.dump(data[600:], outfile)

### Open Words with BIO Notation Label

In [139]:
with open('reviews-aspect-label.json') as f:
    labeled = json.load(f)

display(type(labeled))
data = []
for sentence in labeled:
    sen = []
    for item in sentence:
        tup = (item['token'],item['pos'],item['label'])
        sen.append(tup)
    data.append(sen)
display(data)

list

[[('It', 'PRP', 'O'),
  ('is', 'VBZ', 'O'),
  ('very', 'RB', 'O'),
  ('gud', 'JJ', 'O'),
  ('place', 'NN', 'B'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('food', 'NN', 'O'),
  ('the', 'DT', 'O'),
  ('food', 'NN', 'B'),
  ('is', 'VBZ', 'O'),
  ('too', 'RB', 'O'),
  ('gud', 'JJ', 'O'),
  ('and', 'CC', 'O'),
  ('mouth', 'JJ', 'O'),
  ('water', 'NN', 'O'),
  ('fascinating', 'VBG', 'O'),
  ('quality', 'NN', 'O'),
  ('and', 'CC', 'O'),
  ('quantity', 'NN', 'O'),
  ('that', 'IN', 'O'),
  ('u', 'JJ', 'O'),
  ('ever', 'RB', 'O'),
  ('going', 'VBG', 'O'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('awesome', 'JJ', 'O'),
  ('food', 'NN', 'O'),
  ('really', 'RB', 'O'),
  ('loved', 'VBD', 'O'),
  ('it', 'PRP', 'O')],
 [('Great', 'NNP', 'O'),
  ('food', 'NN', 'B'),
  (',', ',', 'O'),
  ('indoor', 'NN', 'O'),
  ('and', 'CC', 'O'),
  ('outdoor', 'NN', 'O'),
  ('seating', 'NN', 'O')],
 [('Loved', 'VBN', 'O'),
  ('all', 'DT', 'O'),
  ('our', 'PRP$', 'O'),
  ('food', 'NN', 'B'),
  (',', ',', 'O')

In [140]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    features = [
        'bias',
        'word.lower=' + word.lower(),
        'postag=' + postag
    ]

    # Fitur untuk kata non awal kalimat
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS') # Tanda awal kalimat

    # Fitur untuk kata non akhir kalimat
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS') #tanda akhir kalimat
    return features

In [116]:
feat = word2features(train[2],1)
print(feat)

['bias', 'word.lower=all', 'postag=DT', '-1:word.lower=loved', '-1:postag=VBN', '+1:word.lower=our', '+1:postag=PRP$']


In [163]:
from sklearn.model_selection import train_test_split

def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [159]:
display(X_test)
display(y_test)

[[['bias',
   'word.lower=great',
   'postag=NNP',
   'BOS',
   '+1:word.lower=food',
   '+1:postag=NN'],
  ['bias',
   'word.lower=food',
   'postag=NN',
   '-1:word.lower=great',
   '-1:postag=NNP',
   '+1:word.lower=,',
   '+1:postag=,'],
  ['bias',
   'word.lower=,',
   'postag=,',
   '-1:word.lower=food',
   '-1:postag=NN',
   '+1:word.lower=indoor',
   '+1:postag=NN'],
  ['bias',
   'word.lower=indoor',
   'postag=NN',
   '-1:word.lower=,',
   '-1:postag=,',
   '+1:word.lower=and',
   '+1:postag=CC'],
  ['bias',
   'word.lower=and',
   'postag=CC',
   '-1:word.lower=indoor',
   '-1:postag=NN',
   '+1:word.lower=outdoor',
   '+1:postag=NN'],
  ['bias',
   'word.lower=outdoor',
   'postag=NN',
   '-1:word.lower=and',
   '-1:postag=CC',
   '+1:word.lower=seating',
   '+1:postag=NN'],
  ['bias',
   'word.lower=seating',
   'postag=NN',
   '-1:word.lower=outdoor',
   '-1:postag=NN',
   'EOS']],
 [['bias',
   'word.lower=it',
   'postag=PRP',
   'BOS',
   '+1:word.lower=is',
   '+1:pos

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [160]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 0.1, #L1 penalty
    'c2': 0.01, #L2 penalty
    'max_iterations': 200,
    'feature.possible_transitions': True
})
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 58
Seconds required: 0.002

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 4.843177
Feature norm: 1.000000
Error norm: 3.789306
Active features: 58
Line search trials: 1
Line search step: 0.122006
Seconds required for this iteration: 0.000

***** Iteration #2 *****
Loss: 4.021040
Feature norm: 0.991906
Error norm: 2.580315
Active features: 53
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 2.831057
Feature norm: 1.720565
Error norm: 2.368734
Active features: 58
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.001

***** Iterat

### Get y prediction

In [166]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

loved (O)
all (O)
our (O)
food (B)
, (O)
licensed (O)
, (O)
fast (O)
, (O)
efficient (O)
service (B)
[['O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'B'], ['O', 'O', 'O', 'O', 'B', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O']]
[['O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'B'], ['O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [170]:
i = 0
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))
print(y_pred)
print(y_test)

loved (O)
all (O)
our (O)
food (B)
, (O)
licensed (O)
, (O)
fast (O)
, (O)
efficient (O)
service (B)
[['O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'B'], ['O', 'O', 'O', 'O', 'B', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O']]
[['O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'B'], ['O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


### Classification Score

In [169]:
from sklearn.metrics import classification_report
labels = {"B": 0, "I": 1,"O":2}

truth = np.array([labels[bio] for sentence in y_test for bio in sentence])
prediction = np.array([labels[bio] for sentence in y_pred for bio in sentence])

target_names = ['B', 'O']
print(classification_report(truth, prediction, target_names=target_names))

[2 2 2 0 2 2 2 2 2 2 0 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2]
[2 2 2 0 2 2 2 2 2 2 0 2 2 2 2 0 2 2 0 2 0 2 2 2 2 2 0 2 0 2 0 2 2 2 2 2 2
 2 0 2 2 2]
              precision    recall  f1-score   support

           B       0.44      1.00      0.62         4
           O       1.00      0.87      0.93        38

   micro avg       0.88      0.88      0.88        42
   macro avg       0.72      0.93      0.77        42
weighted avg       0.95      0.88      0.90        42

